In [46]:
import tensorcircuit as tc
import numpy as np

### oracle门接口
需要实现`oracle(c,Measure)`，其中`c`为待接入的量子电路，`Measure`为`False`时为将满足条件的状态取反，为`True`时为找到满足条件的概率。

In [47]:
def oracle1(c,Measure):
    for i in range(1,6): c.CNOT(0,i)
    c.multicontrol(2,3,4,5,6,ctrl=[0,1,0,1],unitary=tc.gates.x())
    if Measure: return c.measure(6, with_prob = True)[0][0]
    c.z(6)
    c.multicontrol(2,3,4,5,6,ctrl=[0,1,0,1],unitary=tc.gates.x())
    for i in range(1,6): c.CNOT(0,i)

In [3]:
def reflection(n,c):
    for i in range(n):
        c.H(i)
        c.X(i)
    c.multicontrol(*range(n), unitary=tc.gates.z(), ctrl=[1 for _ in range(n - 1)])
    for i in range(n):
        c.X(i)
        c.H(i)
    return c

### 支持解的个数未知的grover搜索

如果有$t$个答案，一共有$N=2^n$种可能的状态。

则令$\theta=\arcsin(\sqrt{\dfrac TN})$，进行$k$次oracle&reflection操作，其中$k$在$[0,m)$中随机选择，正确率为：

$$
\begin{align}
P(m)=&\dfrac1m\sum_{k=0}^{m-1}\sin^2 (2k+1)\theta\\
=&\dfrac1{2m}\sum_{k=0}^{m-1}1-\cos(4k+2)\theta\\
=&\dfrac 12-\dfrac1{2m}\dfrac{\sin(4m\theta)}{2\sin(2\theta)}
\end{align}
$$

则$m\ge \sin(2\theta)^{-1}\Rightarrow P(m)>\dfrac 14$。

取$m=\sqrt N$即可在$O(\sqrt N)$的时间复杂度内求解。

按以下步骤可以做到$O(\sqrt{\dfrac Nt})$：

1. 令$m=1,\lambda\in(1,\dfrac 43)$。
2. 执行上述步骤，如果是解，则返回。
3. 令$m\leftarrow \min(\sqrt N,\lambda m)$，重复执行2.

证明：

1. $m<\sqrt{\dfrac Nt}$时，则执行时间为$O(\sqrt{\dfrac Nt})\sum_{i\ge 0}\lambda^{-i}=O(\sqrt{\dfrac Nt})$
2. $m\ge\sqrt{\dfrac Nt}$时，执行时间为$\sum_{i\ge 0}\lambda^i(\dfrac 43)^{-i}\sqrt{\dfrac Nt}=O(\sqrt{\dfrac Nt})$

In [48]:
from random import randint
def grover(n,m,oracle):
    def grover_unit(n,m,oracle,iterations):
        c=tc.Circuit(n+m)
        for i in range(n):
            c.H(i)
        for i in range(iterations):
            oracle(c,False)
            reflection(n,c)
        return c.measure(*range(n), with_prob = True)
    
    mx,mul,run_time=1,1.2,0
    while True:
        run_time+=1
        q=grover_unit(n,m,oracle,randint(0,int(mx)))[0]
#        q=grover_unit(n,m,oracle,int(2**(n/2)*np.pi/4))[0]
        c=tc.Circuit(n+m)
        for i in range(n):
            if(q[i]>0.5): c.X(i)
        if(oracle(c,True)>0.5):
            print(q)
            print(run_time)
            return
        mx=min(mx*mul,2**(n/2))
        
        
grover(6,1,oracle1)

[0. 0. 0. 1. 0. 1.]
1
